In [0]:
import pandas as pd
import numpy as np
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/HAR Dataset/HAR Mine")

#os.chdir("/content/drive/My Drive/ActivityChallenge/FinalData")

In [4]:
!ls "/content/drive/My Drive/HAR Dataset/HAR Mine"
#!cp "/content/drive/My Drive/HAR Dataset/HAR Mine/Basic Data/accelerometer_data_merged_all.csv" "accelerometer_data_merged_all.csv"

'Additional Testing Code'
'Basic Data'
 checkpoint
'Classifier Result.gsheet'
 converted_model.tflite
 frozen_har.pb
 history.p
 human_activity_recognition_basic.ipynb
 human_activity_recognition_Classifier.ipynb
 Human_activity_recognition_feature_extract.ipynb
 human_activity_recognition_MINE_2.ipynb
 human_activity_recognition_MINE.ipynb
 human_activity_recognition_with_raw_data.ipynb
 predictions.p
'Processed Data'
'RNN Result.gsheet'
'Subject with number activities performed.gsheet'
 wisdomDataRemoveJoggingActivity.ipynb


# Load Data

In [0]:
#df = pd.read_csv('Basic Data/accelerometer_data_merged_all.csv')
df = pd.read_csv('Basic Data/accelerometer_data_merged_all_with_magnitude.csv')

In [6]:
print(df.head())

          x          y         z  ...               class  timestamp_s  magnitude
0 -0.304352   3.244934 -3.266479  ...  WALKING_DOWNSTAIRS   1563706375   4.614338
1 -0.725739   4.875382 -2.603287  ...  WALKING_DOWNSTAIRS   1563706375   5.574329
2 -5.396820   4.272049 -0.944107  ...  WALKING_DOWNSTAIRS   1563706375   6.947475
3 -4.520538   2.160355 -2.248947  ...  WALKING_DOWNSTAIRS   1563706375   5.491827
4 -7.034454  17.066650  2.139618  ...  WALKING_DOWNSTAIRS   1563706375  18.583112

[5 rows x 8 columns]


In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151329 entries, 0 to 151328
Data columns (total 8 columns):
x               151329 non-null float64
y               151329 non-null float64
z               151329 non-null float64
timestamp_ms    151329 non-null float64
subject_id      151329 non-null int64
class           151329 non-null object
timestamp_s     151329 non-null int64
magnitude       151329 non-null float64
dtypes: float64(5), int64(2), object(1)
memory usage: 9.2+ MB
None


#  Extract feature

In [0]:
def mean(df, columnNames):
  return np.asarray(np.mean(df[columnNames]))

In [0]:
def std(df, columnNames):
  return np.asarray(np.std(df[columnNames]))

In [0]:
def energy(df, columnNames):
  return np.asarray((df[columnNames]**2).mean())

In [0]:
def rms(df, columnNames):
  return np.asarray((df[columnNames]**2).mean()**0.5)

In [0]:
from scipy.stats import iqr
def IQR(df, columnNames):
  return np.asarray(iqr(df[columnNames], rng=(25,75), interpolation='midpoint', axis = 0))

In [0]:
def min_max(df, columnNames):
  return np.asarray(df[columnNames].min()), np.asarray(df[columnNames].max()) 

In [0]:
from scipy.stats import kurtosis, skew

def Skew(df, columnNames):
  return skew(df[columnNames],axis=0)

def Kurtosis(df, columnNames):
  return kurtosis(df[columnNames],axis=0)

In [15]:
'''
import pandas as pd
import numpy as np
  
# Creating the dataframe

df = pd.DataFrame({"A":[12, 4, 5, 44, 1], 
                   "B":[5, 2, 54, 3, 2], 
                   "C":[20, 16, 7, 3, 8], 
                   "D":[14, 3, 17, 2, 6]}) 
#print(df)

a= np.asarray(df[['A','B','C']].kurt(axis = 0))
print (type(a),a)
'''

'\nimport pandas as pd\nimport numpy as np\n  \n# Creating the dataframe\n\ndf = pd.DataFrame({"A":[12, 4, 5, 44, 1], \n                   "B":[5, 2, 54, 3, 2], \n                   "C":[20, 16, 7, 3, 8], \n                   "D":[14, 3, 17, 2, 6]}) \n#print(df)\n\na= np.asarray(df[[\'A\',\'B\',\'C\']].kurt(axis = 0))\nprint (type(a),a)\n'

In [16]:
import numpy as np
def correlation(x, y):
    std_x = (x - x.mean()) / x.std(ddof=0)
    std_y = (y - y.mean()) / y.std(ddof=0)
    
    print(type(std_x),std_x.shape,std_x)
    print('....',type(std_y),std_y)
    mm = (std_x * std_y).mean()
    print(type(mm), mm)
    
    return (std_x * std_y).mean()

#x = y =  np.arange(0.0,5.0,1.0)
x =  np.asarray([-1,2,-3,4])
y =   np.asarray([2,-6,1,4])
print(correlation(x, y))

<class 'numpy.ndarray'> (4,) [-0.55708601  0.55708601 -1.29986737  1.29986737]
.... <class 'numpy.ndarray'> [ 0.46460632 -1.65930828  0.19911699  0.99558497]
<class 'numpy.float64'> -0.03697509744106492
-0.03697509744106492


In [0]:
def median(df, columnNames):
  return np.asarray(df[columnNames].median())

In [0]:
def mad(df, columnNames):
  return  np.asarray(df[columnNames].mad(axis = 0)) 

In [0]:
def corr(df, col1, col2):
  return  df[col1].corr(df[col2])

In [20]:
def magnitude(df, columnNames):
  return np.linalg.norm(df[columnNames].values,axis=1)


'''
d = { 'Id': [1, 2, 1,3],
          'Age':[20, 21, 19, 18]}
 
# Create DataFrame
ddd = pd.DataFrame(d)
print(ddd)
x = np.asarray([1, 2, 1,3])
y =  np.asarray([20, 21, 19, 18])

print(correlation(x, y))
print(corr(ddd, 'Id' ,'Age'))



data = { 'Id': [1, 2, 1,3],
          'Age':[20, 21, 19, 18]}
 
# Create DataFrame
df = pd.DataFrame(data)

#df['aa'] = np.asarray((df[['Id','Age']]**2).sum(axis=1))

#b = np.asarray(np.sqrt(np.square(df[['Id','Age']]).sum(axis=1)))
#print(b, type(b))

# calculate magnitude
df['bb'] = np.sqrt(np.square(df[['Id','Age']]).sum(axis=1))
print(df)

'''

"\nd = { 'Id': [1, 2, 1,3],\n          'Age':[20, 21, 19, 18]}\n \n# Create DataFrame\nddd = pd.DataFrame(d)\nprint(ddd)\nx = np.asarray([1, 2, 1,3])\ny =  np.asarray([20, 21, 19, 18])\n\nprint(correlation(x, y))\nprint(corr(ddd, 'Id' ,'Age'))\n\n\n\ndata = { 'Id': [1, 2, 1,3],\n          'Age':[20, 21, 19, 18]}\n \n# Create DataFrame\ndf = pd.DataFrame(data)\n\n#df['aa'] = np.asarray((df[['Id','Age']]**2).sum(axis=1))\n\n#b = np.asarray(np.sqrt(np.square(df[['Id','Age']]).sum(axis=1)))\n#print(b, type(b))\n\n# calculate magnitude\ndf['bb'] = np.sqrt(np.square(df[['Id','Age']]).sum(axis=1))\nprint(df)\n\n"

In [21]:
#print(df.isnull().any().any())
#df = df.fillna(method='ffill').astype('float64')
df = df.fillna(method='ffill')
#print(df['x'])

# FOR MAGNITUDE Calculation
#df['magnitude'] = magnitude(df, ['x','y','z'])

print(df.head())
print (len(df))
#print(df.isnull().any().any())
#df.to_csv('Basic Data/accelerometer_data_merged_all_with_magnitude.csv', index=False)

          x          y         z  ...               class  timestamp_s  magnitude
0 -0.304352   3.244934 -3.266479  ...  WALKING_DOWNSTAIRS   1563706375   4.614338
1 -0.725739   4.875382 -2.603287  ...  WALKING_DOWNSTAIRS   1563706375   5.574329
2 -5.396820   4.272049 -0.944107  ...  WALKING_DOWNSTAIRS   1563706375   6.947475
3 -4.520538   2.160355 -2.248947  ...  WALKING_DOWNSTAIRS   1563706375   5.491827
4 -7.034454  17.066650  2.139618  ...  WALKING_DOWNSTAIRS   1563706375  18.583112

[5 rows x 8 columns]
151329


In [22]:
# FOR MAGNITUDE Calculation
#df['magnitude'] = magnitude(df, ['x','y','z'])
print(df.head())
print (len(df))
#df.to_csv('Basic Data/accelerometer_data_merged_all_with_magnitude.csv', index=False)

          x          y         z  ...               class  timestamp_s  magnitude
0 -0.304352   3.244934 -3.266479  ...  WALKING_DOWNSTAIRS   1563706375   4.614338
1 -0.725739   4.875382 -2.603287  ...  WALKING_DOWNSTAIRS   1563706375   5.574329
2 -5.396820   4.272049 -0.944107  ...  WALKING_DOWNSTAIRS   1563706375   6.947475
3 -4.520538   2.160355 -2.248947  ...  WALKING_DOWNSTAIRS   1563706375   5.491827
4 -7.034454  17.066650  2.139618  ...  WALKING_DOWNSTAIRS   1563706375  18.583112

[5 rows x 8 columns]
151329


In [23]:
unisubs = np.unique(df['subject_id'])
print(unisubs, type(unisubs))

[1 2 3 4 5 6 7 8] <class 'numpy.ndarray'>


**Subject ID 1** Atik WALKING_UPSTAIRS activity missing

**Subject ID 3** Niraj walking_downstairs and sitting activity missing



**Subject ID**   >>>>  **Name**

1                     Atik 

2                     Hasan

3                     Niraj

4                     Mahir

5                     Jamil

6                     Tushar

7                     Uddip   

8                     Obaidur

In [0]:
def processAccelerometerWithConsecutiveActivities(window_size=50, overlapping = 1):
  overlapping_count = int(overlapping* window_size)
  data = df.values
  #uniseg = np.unique(df1['segment'])
  #unisubs = np.unique(df['subject_id'])
  
  unisubs = np.asarray([2,4,5,6,7,8])
  
  # Need to change when column adds or removes
  columns=['mean_x', 'mean_y', 'mean_z', 'magnitude_mean',
            'std_x', 'std_y', 'std_z', 'magnitude_std',
            'rms_x', 'rms_y', 'rms_z', 'magnitude_rms',
            'min_x', 'min_y', 'min_z', 'magnitude_min',
            'max_x', 'max_y', 'max_z', 'magnitude_max',
            'median_x', 'median_y', 'median_z', 'magnitude_median',
            'mad_x', 'mad_y', 'mad_z','magnitude_mad',
           'corr_xy','corr_yz', 'corr_xz',
            'subject',  'class']

  accdataFrame = pd.DataFrame(columns = columns)
  cols = ['x', 'y', 'z','magnitude']

  for i in range(len(unisubs)):
    one_subject_segdata = df.loc[df['subject_id'] == unisubs[i]]
    print('Subject ', unisubs[i], ", Activities Found\n",np.unique(one_subject_segdata['class']))
    #one_subject_unique_activities = np.unique(one_subject_segdata['activity'])
    
    if(len(one_subject_segdata) >0 ):
      activity = one_subject_segdata['class'].values[0]
      start = 0
      
      for j in range (0, len(one_subject_segdata)):
        if(one_subject_segdata['class'].values[j] == activity):
          #print ('in if condition start is ', start, ' end is ', end , ' activity ', one_subject_segdata['Activity'].values[i])
          continue
        else:
          one_subject_one_activity_data = one_subject_segdata[start: j]
          #print(one_subject_one_activity_data,"\n------------------")
          totalwindow = math.ceil(len(one_subject_one_activity_data)/window_size)
          
          start2 =0 
          for k in range (0, len(one_subject_one_activity_data) - window_size, overlapping_count):
            start2 = k
            windowdata = one_subject_one_activity_data[k: k+window_size]
            means = mean(windowdata, cols)
            stds = std(windowdata, cols)
            rmses = rms(windowdata, cols)
            mins, maxs = min_max(windowdata, cols)
            medians = median(windowdata, cols)
            mads = mad(windowdata, cols)
            corr_xy = corr(windowdata, 'x', 'y')
            corr_yz = corr(windowdata, 'y', 'z')
            corr_xz = corr(windowdata, 'x', 'z')
            
            
            #energies = energy(windowdata, cols)
            #iqrs = IQR(windowdata, cols)
            '''
            if windowdata.shape[0]==0:
              skews = skews
              kurts = kurts
            else:
              skews = Skew(windowdata, cols)
              kurts = Kurtosis(windowdata, cols)
            '''
            
            res = [means[0], means[1], means[2], means[3],
                   stds[0], stds[1], stds[2], stds [3],
                   rmses[0], rmses[1], rmses[2], rmses[3],
                   mins[0], mins[1], mins[2], mins[3],
                   maxs[0], maxs[1], maxs[2], maxs[3],
                   medians[0], medians[1], medians[2], medians[3],
                   mads[0], mads[1], mads[2], mads[3],
                   corr_xy, corr_yz, corr_xz,
                   one_subject_one_activity_data['subject_id'].values[0], one_subject_one_activity_data['class'].values[0] ]
            accdataFrame.loc[len(accdataFrame)] = res
          
          if not start2 == len(one_subject_one_activity_data):
            windowdata = one_subject_one_activity_data[start2 : len(one_subject_one_activity_data)]
            means = mean(windowdata, cols)
            stds = std(windowdata, cols)
            rmses = rms(windowdata, cols)
            mins, maxs = min_max(windowdata, cols)
            medians = median(windowdata, cols)
            mads = mad(windowdata, cols)
            corr_xy = corr(windowdata, 'x', 'y')
            corr_yz = corr(windowdata, 'y', 'z')
            corr_xz = corr(windowdata, 'x', 'z')
            
            
            '''
            if windowdata.shape[0]==0:
              skews = skews
              kurts = kurts
            else:
              skews = Skew(windowdata, cols)
              kurts = Kurtosis(windowdata, cols)
            '''  
            
            res = [means[0], means[1], means[2], means[3],
                   stds[0], stds[1], stds[2], stds [3],
                   rmses[0], rmses[1], rmses[2], rmses[3],
                   mins[0], mins[1], mins[2], mins[3],
                   maxs[0], maxs[1], maxs[2], maxs[3],
                   medians[0], medians[1], medians[2], medians[3],
                   mads[0], mads[1], mads[2], mads[3],
                   corr_xy, corr_yz, corr_xz,
                   one_subject_one_activity_data['subject_id'].values[0], one_subject_one_activity_data['class'].values[0] ]


            accdataFrame.loc[len(accdataFrame)] = res
          
          start = j
          activity = one_subject_segdata['class'].values[j]
          #print ('in else condition start is ', start, ' end is ', end, ' activity ', one_subject_segdata['Activity'].values[i])
          #print('In else ', activity, start)


      if not start == len(one_subject_segdata):
        #print(one_subject_segdata[start: len(one_subject_segdata)])
        # need to do some code here
        one_subject_one_activity_data = one_subject_segdata[start: len(one_subject_segdata)]
        #print(one_subject_one_activity_data,"\n------------------")
        totalwindow = math.ceil(len(one_subject_one_activity_data)/window_size)

        start2 =0 
        for k in range (0, len(one_subject_one_activity_data) - window_size, overlapping_count):
          start2 = k
          windowdata = one_subject_one_activity_data[k: k+window_size]
          means = mean(windowdata, cols)
          stds = std(windowdata, cols)
          rmses = rms(windowdata, cols)
          mins, maxs = min_max(windowdata, cols)
          medians = median(windowdata, cols)
          mads = mad(windowdata, cols)
          corr_xy = corr(windowdata, 'x', 'y')
          corr_yz = corr(windowdata, 'y', 'z')
          corr_xz = corr(windowdata, 'x', 'z')

          '''
          if windowdata.shape[0]==0:
            skews = skews
            kurts = kurts
          else:
            skews = Skew(windowdata, cols)
            kurts = Kurtosis(windowdata, cols)
          '''
            
          res = [means[0], means[1], means[2], means[3],
                   stds[0], stds[1], stds[2], stds [3],
                   rmses[0], rmses[1], rmses[2], rmses[3],
                   mins[0], mins[1], mins[2], mins[3],
                   maxs[0], maxs[1], maxs[2], maxs[3],
                   medians[0], medians[1], medians[2], medians[3],
                   mads[0], mads[1], mads[2], mads[3],
                   corr_xy, corr_yz, corr_xz,
                   one_subject_one_activity_data['subject_id'].values[0], one_subject_one_activity_data['class'].values[0] ]
          accdataFrame.loc[len(accdataFrame)] = res

        if not start2 == len(one_subject_one_activity_data):
          windowdata = one_subject_one_activity_data[start2 : len(one_subject_one_activity_data)]
          means = mean(windowdata, cols)
          stds = std(windowdata, cols)
          rmses = rms(windowdata, cols)
          mins, maxs = min_max(windowdata, cols)
          medians = median(windowdata, cols)
          mads = mad(windowdata, cols)
          corr_xy = corr(windowdata, 'x', 'y')
          corr_yz = corr(windowdata, 'y', 'z')
          corr_xz = corr(windowdata, 'x', 'z')
          '''
          if windowdata.shape[0]==0:
            skews = skews
            kurts = kurts
          else:
            skews = Skew(windowdata, cols)
            kurts = Kurtosis(windowdata, cols)
          '''
          res = [means[0], means[1], means[2], means[3],
                   stds[0], stds[1], stds[2], stds [3],
                   rmses[0], rmses[1], rmses[2], rmses[3],
                   mins[0], mins[1], mins[2], mins[3],
                   maxs[0], maxs[1], maxs[2], maxs[3],
                   medians[0], medians[1], medians[2], medians[3],
                   mads[0], mads[1], mads[2], mads[3],
                   corr_xy, corr_yz, corr_xz,
                   one_subject_one_activity_data['subject_id'].values[0], one_subject_one_activity_data['class'].values[0] ]

          accdataFrame.loc[len(accdataFrame)] = res
  
  return accdataFrame

In [0]:
def processAccelerometer(window_size=50 , overlapping = 1):
  overlapping_count = int (overlapping* window_size)
  data = df.values
  #unisubs = np.unique(df['subject_id'])
  unisubs = np.asarray([2,4,5,6,7,8])
  print('Number of uniqe subject ', len(unisubs))

  # need to change
  columns=['mean_x', 'mean_y', 'mean_z', 'magnitude_mean',
            'std_x', 'std_y', 'std_z', 'magnitude_std',
            'rms_x', 'rms_y', 'rms_z', 'magnitude_rms',
            'min_x', 'min_y', 'min_z', 'magnitude_min',
            'max_x', 'max_y', 'max_z', 'magnitude_max',
            'median_x', 'median_y', 'median_z', 'magnitude_median',
            'mad_x', 'mad_y', 'mad_z','magnitude_mad',
           'corr_xy','corr_yz', 'corr_xz',
            'subject',  'class']

  accdataFrame = pd.DataFrame(columns = columns)
  cols = ['x', 'y', 'z','magnitude']
  
  

  for i in range(len(unisubs)):
    
    one_subject_segdata = df.loc[df['subject_id'] == unisubs[i]]
    one_subject_unique_activities = np.unique(one_subject_segdata['class'])
    print('Subject ', unisubs[i], ", Activities Found\n",np.unique(one_subject_segdata['class']))
    
    for j in range(len(one_subject_unique_activities)):
      #print(one_subject_unique_activities[j])
      one_subject_one_activity_data = one_subject_segdata.loc[one_subject_segdata['class'] == one_subject_unique_activities[j]]
      totalwindow = math.ceil(len(one_subject_one_activity_data)/window_size)
      start = 0 
      for k in range (0, len(one_subject_one_activity_data) - window_size, overlapping_count):
        start = k
        windowdata = one_subject_one_activity_data[k: k+window_size]
        means = mean(windowdata, cols)
        stds = std(windowdata, cols)
        rmses = rms(windowdata, cols)
        mins, maxs = min_max(windowdata, cols)
        medians = median(windowdata, cols)
        mads = mad(windowdata, cols)
        corr_xy = corr(windowdata, 'x', 'y')
        corr_yz = corr(windowdata, 'y', 'z')
        corr_xz = corr(windowdata, 'x', 'z')
        
        '''
        if windowdata.shape[0]==0:
          skews = skews
          kurts = kurts
        else:
          skews = Skew(windowdata, cols)
          kurts = Kurtosis(windowdata, cols)
        '''
        res = [means[0], means[1], means[2], means[3],
                   stds[0], stds[1], stds[2], stds [3],
                   rmses[0], rmses[1], rmses[2], rmses[3],
                   mins[0], mins[1], mins[2], mins[3],
                   maxs[0], maxs[1], maxs[2], maxs[3],
                   medians[0], medians[1], medians[2], medians[3],
                   mads[0], mads[1], mads[2], mads[3],
                   corr_xy, corr_yz, corr_xz,
                   one_subject_one_activity_data['subject_id'].values[0], one_subject_one_activity_data['class'].values[0] ]
        accdataFrame.loc[len(accdataFrame)] = res
        #print(accdataFrame[['subject','class']])

      if not start == len(one_subject_one_activity_data):
        windowdata = one_subject_one_activity_data[start : len(one_subject_segdata)]
        means = mean(windowdata, cols)
        stds = std(windowdata, cols)
        rmses = rms(windowdata, cols)
        mins, maxs = min_max(windowdata, cols)
        medians = median(windowdata, cols)
        mads = mad(windowdata, cols)
        corr_xy = corr(windowdata, 'x', 'y')
        corr_yz = corr(windowdata, 'y', 'z')
        corr_xz = corr(windowdata, 'x', 'z')
        
        '''
        if windowdata.shape[0]==0:
          skews = skews
          kurts = kurts
        else:
          skews = Skew(windowdata, cols)
          kurts = Kurtosis(windowdata, cols)
        '''
        
        res = [means[0], means[1], means[2], means[3],
                   stds[0], stds[1], stds[2], stds [3],
                   rmses[0], rmses[1], rmses[2], rmses[3],
                   mins[0], mins[1], mins[2], mins[3],
                   maxs[0], maxs[1], maxs[2], maxs[3],
                   medians[0], medians[1], medians[2], medians[3],
                   mads[0], mads[1], mads[2], mads[3],
                   corr_xy, corr_yz, corr_xz,
                   one_subject_one_activity_data['subject_id'].values[0], one_subject_one_activity_data['class'].values[0] ]

        accdataFrame.loc[len(accdataFrame)] = res
        #print(accdataFrame[['subject','class']])
  
  return accdataFrame

In [26]:
accelerometer = processAccelerometerWithConsecutiveActivities(window_size =50)
#accelerometer = processAccelerometer(window_size = 50)

#accelerometer = accelerometer.fillna(method='ffill').astype('float64')
accelerometer = accelerometer.fillna(method='ffill')
print(accelerometer.head(3))

unique_activities = np.unique(accelerometer['class'])
print(unique_activities)

Subject  2 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
Subject  4 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
Subject  5 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
Subject  6 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
Subject  7 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
Subject  8 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
     mean_x    mean_y    mean_z  ...   corr_xz  subject    class
0 -0.386284  8.558812 -2.393511  ...  0.051671        2  WALKING
1  0.539121  9.308389 -0.242513  ...  0.388097        2  WALKING
2  0.367984  9.484555 -0.722646  ...  0.407794        2  WALKING

[3 rows x 33 columns]
['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']


In [27]:
#print(len(df))
print(len(accelerometer))
print(len(df)/50)

2539
3026.58


In [0]:
accelerometer.to_csv('Processed Data/accelerometer_extracted_featured_data_50_window_size_consecutive_activity_without_overlapping.csv', index=False)

In [44]:
#accelerometer = processAccelerometer(window_size = 30)
accelerometer = processAccelerometer(window_size = 30, overlapping = 0.5)

#accelerometer = accelerometer.fillna(method='ffill').astype('float64')
accelerometer = accelerometer.fillna(method='ffill')
print(accelerometer.head(3))

unique_activities = np.unique(accelerometer['class'])
print(unique_activities)

Number of uniqe subject  6
Subject  2 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
Subject  4 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
Subject  5 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
Subject  6 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
Subject  7 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
Subject  8 , Activities Found
 ['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPSTAIRS']
     mean_x    mean_y    mean_z  ...   corr_xz  subject    class
0 -0.774737  8.112980 -3.890574  ...  0.420879        2  SITTING
1 -1.982532  6.981241 -5.287511  ...  0.460519        2  SITTING
2 -3.484655  5.463636 -6.801844  ...  0.788508        2  SITTING

[3 rows x 33 columns]
['SITTING' 'STANDING' 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_UPST

In [45]:
#print(len(df))
print(len(accelerometer))
print(len(df)/50)

8386
3026.58


In [0]:
#accelerometer.to_csv('Processed Data/accelerometer_extracted_featured_data_30_window_size_without_overlapping.csv', index=False)
accelerometer.to_csv('Processed Data/accelerometer_extracted_featured_data_30_window_size_with_50_percent_overlapping.csv', index=False)